## pre-trained된 모델로 예시 만들기

In [ ]:
pip install transformers

In [ ]:
from transformers import GPT2LMHeadModel,PreTrainedTokenizerFast

In [ ]:


tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2")
   # bos_token=BOS,
    #eos_token=EOS,
    #unk_token="<unk>",
    #pad_token=PAD,
    #mask_token=MASK,
    #additional_special_tokens=[
    #    Q_TKN,
    #    A_TKN,
    #    SENT

tokenizer.add_special_tokens({'pad_token': '<pad>','bos_token':'<bos>','eos_token': '<eos>'})
#tokenizer에서 지정한 토큰에 한해서 special token으로 할당 가능
tokenizer.add_tokens(['<bot>'])
#tokenizer에서 지정한 토큰이 아닐경우 따로 지정해줌. 모두 토큰으로 분리됨
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
model.resize_token_embeddings(len(tokenizer))
#tokenizer에 추가해줬기 때문에 tokenizer에 있는 어휘사전 재조정

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. Th

Embedding(51203, 768)

In [ ]:
print(tokenizer.decode(model.generate(**tokenizer('옛날옛적에 ',return_tensors='pt'))[0]))

옛날옛적에 <unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>


## DataSet 구성하기

In [ ]:
from torch.utils.data import Dataset

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('story_mapping+character+order.csv');df

,Unnamed: 0,genre,motif,unit_motif,unit_stage,scene_chracter,scene_emotion,scene_content,narrative_stage,mapped_emotion
0,0,"['판타지', '드라마']",납치,['비밀의 공유'],['Final Salvo'],"[""['C003']"", ""['C008']""]","['담담하다', '단호하다']","['저희 집의 방보다 훨씬 작은데 괜찮으세요?', '딴 소리 말고, 얼른 보여줘요....",결말,['중립']
1,1,"['판타지', '드라마']",납치,['비밀의 공유'],['Final Salvo'],"[""['C003']"", ""['C008']""]","['담담하다', '놀라다']","['C008이 낡은 수첩을 보여준다.', 'C003가 수첩을 본다.', '말도 안 ...",결말,"['관심', '중립']"
2,2,"['판타지', '드라마']",납치,['비밀의 공유'],['Final Salvo'],"[""['C003']"", ""['C008']""]","['궁금하다', '놀라다']","['C003의 말을 듣고 궁금해한다.', '왜 그리 놀라십니까?', 'C003가 수...",결말,['관심']
3,3,"['판타지', '드라마']",납치,['비밀의 공유'],['Final Salvo'],"[""['C003']""]",['놀라다'],['C003는 수첩 마지막 장에 적힌 자신의 이름을 보고 놀란다.'],결말,['관심']
4,4,"['판타지', '드라마']",납치,['공적인 입장표명'],['Opening Salvo'],"[""['중신1']"", ""['C002']""]","['단호하다', '초조하다']","['C002이 발표한다.', 'C001은 역모를 꾸미지 않았다. 그러니 C001은 ...",도입,"['중립', '두려움']"
...,...,...,...,...,...,...,...,...,...,...
298267,129,['멜로/로맨스'],첫눈에 반한 사랑,['회의감'],['Choice to Fight'],"[""['여선생']"", ""['C002']""]","['괴롭다', '담담하다', '안타깝다']","['네 편이야.', 'C002이 집안일을 한다.', 'C002이 설거지를 하며 운다.']",절정,"['중립', '슬픔']"
298268,130,['멜로/로맨스'],첫눈에 반한 사랑,['재회'],['Resurrection'],"[""['C001']"", ""['C002']""]","['씁쓸하다', '반갑다', '긴장되다']","['저기요.', 'C002이 C001에게 인사한다.', 'C001이 C002에게 선...",절정,"['두려움', '슬픔', '행복']"
298269,131,['멜로/로맨스'],첫눈에 반한 사랑,['재회'],['Resurrection'],"[""['C001']""]","['불편하다', '당황하다']","['반가워요.', '잠시만요.', 'C001이 강의실 뒤편을 기웃거린다.']",절정,"['분노', '두려움']"
298270,132,['멜로/로맨스'],첫눈에 반한 사랑,['재회'],['Resurrection'],"[""['아줌마']"", ""['C001']"", ""['C002']""]","['황당하다', '초조하다']","['뒷문 없나요?', '아줌마가 C001에게 뒷문은 없다고 말한다.', 'C002이...",절정,"['분노', '두려움']"


In [ ]:
# 행마다 대괄호 [ ] 제거
df['scene_content1'] = df['scene_content'].str.replace(r"[\[\]]", "", regex=True)
df

,Unnamed: 0,genre,motif,unit_motif,unit_stage,scene_chracter,scene_emotion,scene_content,narrative_stage,mapped_emotion,scene_content1
0,0,"['판타지', '드라마']",납치,['비밀의 공유'],['Final Salvo'],"[""['C003']"", ""['C008']""]","['담담하다', '단호하다']","['저희 집의 방보다 훨씬 작은데 괜찮으세요?', '딴 소리 말고, 얼른 보여줘요....",결말,['중립'],"'저희 집의 방보다 훨씬 작은데 괜찮으세요?', '딴 소리 말고, 얼른 보여줘요.'..."
1,1,"['판타지', '드라마']",납치,['비밀의 공유'],['Final Salvo'],"[""['C003']"", ""['C008']""]","['담담하다', '놀라다']","['C008이 낡은 수첩을 보여준다.', 'C003가 수첩을 본다.', '말도 안 ...",결말,"['관심', '중립']","'C008이 낡은 수첩을 보여준다.', 'C003가 수첩을 본다.', '말도 안 돼..."
2,2,"['판타지', '드라마']",납치,['비밀의 공유'],['Final Salvo'],"[""['C003']"", ""['C008']""]","['궁금하다', '놀라다']","['C003의 말을 듣고 궁금해한다.', '왜 그리 놀라십니까?', 'C003가 수...",결말,['관심'],"'C003의 말을 듣고 궁금해한다.', '왜 그리 놀라십니까?', 'C003가 수첩..."
3,3,"['판타지', '드라마']",납치,['비밀의 공유'],['Final Salvo'],"[""['C003']""]",['놀라다'],['C003는 수첩 마지막 장에 적힌 자신의 이름을 보고 놀란다.'],결말,['관심'],'C003는 수첩 마지막 장에 적힌 자신의 이름을 보고 놀란다.'
4,4,"['판타지', '드라마']",납치,['공적인 입장표명'],['Opening Salvo'],"[""['중신1']"", ""['C002']""]","['단호하다', '초조하다']","['C002이 발표한다.', 'C001은 역모를 꾸미지 않았다. 그러니 C001은 ...",도입,"['중립', '두려움']","'C002이 발표한다.', 'C001은 역모를 꾸미지 않았다. 그러니 C001은 더..."
...,...,...,...,...,...,...,...,...,...,...,...
298267,129,['멜로/로맨스'],첫눈에 반한 사랑,['회의감'],['Choice to Fight'],"[""['여선생']"", ""['C002']""]","['괴롭다', '담담하다', '안타깝다']","['네 편이야.', 'C002이 집안일을 한다.', 'C002이 설거지를 하며 운다.']",절정,"['중립', '슬픔']","'네 편이야.', 'C002이 집안일을 한다.', 'C002이 설거지를 하며 운다.'"
298268,130,['멜로/로맨스'],첫눈에 반한 사랑,['재회'],['Resurrection'],"[""['C001']"", ""['C002']""]","['씁쓸하다', '반갑다', '긴장되다']","['저기요.', 'C002이 C001에게 인사한다.', 'C001이 C002에게 선...",절정,"['두려움', '슬픔', '행복']","'저기요.', 'C002이 C001에게 인사한다.', 'C001이 C002에게 선생..."
298269,131,['멜로/로맨스'],첫눈에 반한 사랑,['재회'],['Resurrection'],"[""['C001']""]","['불편하다', '당황하다']","['반가워요.', '잠시만요.', 'C001이 강의실 뒤편을 기웃거린다.']",절정,"['분노', '두려움']","'반가워요.', '잠시만요.', 'C001이 강의실 뒤편을 기웃거린다.'"
298270,132,['멜로/로맨스'],첫눈에 반한 사랑,['재회'],['Resurrection'],"[""['아줌마']"", ""['C001']"", ""['C002']""]","['황당하다', '초조하다']","['뒷문 없나요?', '아줌마가 C001에게 뒷문은 없다고 말한다.', 'C002이...",절정,"['분노', '두려움']","'뒷문 없나요?', '아줌마가 C001에게 뒷문은 없다고 말한다.', 'C002이 ..."


In [27]:
class NovelData(Dataset):
    def __init__(self, tokenizer):
        self.data = df
        self.X = []
        for i in self.data['scene_content1']:
          self.X.append(i)

        for idx, i in enumerate(self.X):
          try:
            self.X[idx] = "<bos>" +i + '<bot>: ' +self.X[idx+1]+'<eos>'
          except:
            break

        self.X = self.X[:50000]
        # 데이터셋 개수 정하기

        print(self.X[0])
        #예시로 잘 작동하는지 확인

        self.tokenizer = tokenizer

        self.X_encoded = tokenizer(self.X,padding=True)
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']



    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.tensor(self.input_ids[idx]), torch.tensor(self.attention_mask[idx])
#텐서로 변환하지 않으면 list는 shape이 없다고 오류 뜸


In [ ]:
tokenizer('안녕 오늘 어때?')
#token_type_ids이 경우 문장 구분용으로 0과 1로 구분되며 gpt 기반 모델에선 쓰이지 않음
#attention_mask의 경우 padding과 구분하기 위함이며 padding이 아닌경우 1로 표기되고 아닌경우 0으로 표기됨

{'input_ids': [25906, 10070, 9105, 7312, 406], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [ ]:
noveldata = NovelData(tokenizer)
#문장 구성이 잘 되어있는지 확인용

<bos>'저희 집의 방보다 훨씬 작은데 괜찮으세요?', '딴 소리 말고, 얼른 보여줘요.', 'C003를 바라본다.'<bot>: 'C008이 낡은 수첩을 보여준다.', 'C003가 수첩을 본다.', '말도 안 돼. 이게 왜 여기에 있어? 이게 화타의 유물이란 말이에요?'<eos>


KeyboardInterrupt: 

## 학습시키기

In [ ]:
from torch.optim import Adam
from torch.utils.data import DataLoader
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm

In [ ]:
import torch

In [ ]:
'''def train(noveldata,model,optim):
   epochs = 3

   for i in tqdm(range(epochs)):
    for X, a in noveldata:
      optim.zero_grad()
      X=X.to(device)
      a=a.to(device)
      loss = model(X, attention_mask=a, labels=X).loss
      loss.backward()
      optim.step()

   torch.save(model.state_dict(), 'model_state.pt')
   print(infer("hello how are you"))'''

#훈련시키기
def train(noveldata, model, optim):
    epochs = 3

    for i in range(epochs):

        for X, a in noveldata:
            optim.zero_grad()
            X = X.to(device)
            a = a.to(device)
            loss = model(X, attention_mask=a, labels=X).loss
            loss.backward()
            optim.step()
        print('epoch finished')
    torch.save(model.state_dict(), 'model_state.pt')
    print(infer("안녕 뭐해?"))


# 예측하기
def infer(inp):
    inp = "<bos>"+inp+" <bot>: "
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    a = inp["attention_mask"].to(device)
    output = model.generate(X, attention_mask=a )
    output = tokenizer.decode(output[0])
    return output

device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

model = model.to(device)

noveldata = NovelData(tokenizer)
noveldata = DataLoader(noveldata, batch_size=16)

model.train()

optim = Adam(model.parameters(), lr=1e-5)

print("training...")

train(noveldata,model,optim)

<bos>'저희 집의 방보다 훨씬 작은데 괜찮으세요?', '딴 소리 말고, 얼른 보여줘요.', 'C003를 바라본다.'<bot>: 'C008이 낡은 수첩을 보여준다.', 'C003가 수첩을 본다.', '말도 안 돼. 이게 왜 여기에 있어? 이게 화타의 유물이란 말이에요?'<eos>
training...
epoch finished
